## Predicted AMPs data process

| Step | Description | # of Proteins | # of AMPs |
| - | - | - | - |
| 1. | Combine all predicted-AMPs. | 5,641,747 pro. | 5,764,503 seg. |
| 2. | Protein sequence clustering (cd-hit 0.9-0.8). | - | - |
| 3. | Get cluster-IDs of protein-level and AMP-level. | 466,894 clst. | 458,371 clst. |
| 4. | Get eggnog-mapper annotation. | - | - | 
| 5. | Statistics on AMP- & Pro- clusters. | 183,337 clst. | 94,148 clst. |

In [1]:
import os, re
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option("display.max_columns", None)

#### Step 1, Combine all predicted-AMPs

In [ ]:
### cat input.pred_list.txt
"""
predictions/ancient_human_gut/pred.csv
predictions/BGI_human_oral/pred.csv
predictions/data_CGMR/pred.csv
predictions/data_Hadza/Hadza.Bact.pred
predictions/data_Hadza/Hadza.Phage.pred
predictions/MGnify_cow_rumen/pred.csv
predictions/MGnify_fish_gut/MGnify_fish_gut.pred.tsv
predictions/MGnify_human_gut/pred.csv
predictions/MGnify_human_oral/pred.csv
predictions/MGnify_pig_gut/pred.csv
predictions/MGnify_zibrafish_fecal/MGnify_zibrafish_fecal.pred.tsv
"""

data = pd.DataFrame(columns=['Source','ProID', 'AMP', 'AMPlen', 'Position', 'Sequence'])

with open('input.pred_list.txt') as lst:
    for f in lst.readlines():
        source = f.strip().split('/')[1]
        df = pd.read_csv(f.strip(), sep='\t')
        df['Source'] = source
        data = pd.concat([data,df], ignore_index=True)

data = data[data.ProID!='ProID'] 
data['AMPlen'] = data.AMPlen.astype('int').tolist()
data = data[data.AMPlen>=5]
data['ID'] = data['Source'] + '_' + data['ProID']

In [64]:
print(data.shape)
data.head()

(5764503, 7)


,Source,ProID,AMP,AMPlen,Position,Sequence,ID
0,ancient_human_gut,AZ107.k141_184032_5,FRRKKW,6,"276,281",MQLVLPFEHMAKAAQYMEPSVFEYVREGEIESFESFEKFDFLAFDW...,ancient_human_gut_AZ107.k141_184032_5
1,ancient_human_gut,AZ107.k141_604685_1,VVGVVSRVTNK,11,"15,25",MGLNLLLAGLAVLLVVVGVVSRVTNK,ancient_human_gut_AZ107.k141_604685_1
2,ancient_human_gut,AZ107.k141_630992_1,KKLADAVKGFLDKIFNKK,18,"46,63",MKTSLKRIIAFVLVLALSFAAFATVVSAAPVSASAKTESASIGSFF...,ancient_human_gut_AZ107.k141_630992_1
3,ancient_human_gut,AZ107.k141_368093_2,KKFGKAAN,8,"221,228",MHLMSNSKHKTGRKPLKLGTVENYHHHWYLLIWLVYLTLFAIAEHV...,ancient_human_gut_AZ107.k141_368093_2
4,ancient_human_gut,AZ107.k141_473258_2,YIWKIIK,7,"23,29",MGNKDKNIRLTFYVVCGLLLGAPYIWKIIKLIPELLKTLPNAAEIL...,ancient_human_gut_AZ107.k141_473258_2


#### Step 2, Protein & AMPs sequence clustering (cd-hit 0.9-0.8)

output_path = 'Analysis/cdhit' & 'Analysis/amp_cdhit'

In [ ]:
## output all AMP-contained proteins as .faa
data_dict = dict( zip(data.ID.tolist(), data.Sequence.tolist()) )
with open('all_amp_protein.animal.faa', 'w') as f:
    for name,seq in data_dict.items():
        f.write('>%s\n%s\n' % (name,seq))

## output AMP-contained proteins with length no greater than 10 as short.faa
with open('Analysis/all_amp_protein.animal.short.faa', 'w') as f:
    for name,seq in data_dict.items():
        if len(seq) <= 10:
            f.write('>%s\n%s\n' % (name,seq))

##### Run cd-hit on protein sequence (0.9-0.8)

cd-hit -i ../all_amp_protein.animal.faa -o pro90 -c 0.9 -n 5 -d 0 -M 4500

clstr_sort_by.pl < pro90.clstr no > pro90.sorted.clstr

bioawk -cfastx '{print $name,$seq}' pro90 > pro90.tab


cd-hit -i ./pro90 -o pro80 -c 0.8 -n 5 -d 0 -M 4500

clstr_sort_by.pl < pro80.clstr no > pro80.sorted.clstr

bioawk -cfastx '{print $name,$seq}' pro80 > pro80.tab

##### Run cd-hit on short protein sequence (0.9-0.8)

cd-hit -i ../all_amp_protein.animal.short.faa -o pro90.short -c 0.9 -n 2 -d 0 -l 4

clstr_sort_by.pl < pro90.short.clstr no > pro90.short.sorted.clstr

bioawk -cfastx '{print $name,$seq}' pro90.short > pro90.short.tab


cd-hit -i ./pro90.short -o pro80.short -c 0.8 -n 2 -d 0 -l 4

clstr_sort_by.pl < pro80.short.clstr no > pro80.short.sorted.clstr

bioawk -cfastx '{print $name,$seq}' pro80.short > pro80.short.tab

In [68]:
## output all predicted-AMPs as .faa
def df2faa(df, outfaa):
    ids = df['AMPID'].tolist()
    seqs = df['AMP'].tolist()
    with open(outfaa, 'w') as ofa:
        for i in range(len(ids)):
            ofa.write('>%s\n%s\n' % (ids[i], seqs[i]))

data['AMPID'] = data['ID'] + ':' + data['Position']
data_long = data[data.AMPlen>10]
data_short = data[data.AMPlen<=10]

df2faa(data_long, 'Analysis/amp_cdhit/AMPs.faa')
df2faa(data_short, 'Analysis/amp_cdhit/AMPs.short.faa')

##### Run cd-hit on AMP sequence (0.95)

cd-hit -i AMPs.faa -o amp95 -c 0.95 -n 5 -d 0

clstr_sort_by.pl < amp95.clstr no > amp95.sorted.clstr

##### Run cd-hit on short AMP sequence (0.95)
cd-hit -i AMPs.short.faa -o amp95.short -c 0.95 -n 2 -d 0 -l 4 -M 20000

clstr_sort_by.pl < amp95.short.clstr no > amp95.short.sorted.clstr

#### Step 3, Get cluster-IDs of protein-level and AMP-level

In [2]:
## function to process cd-hit sorted-clstering results

def get_cluster_id(sorted_clstr,start_clst=0):
    # clst_id = {}
    pro_cid = {}
    clst_rep = {}
    cid = 0
    with open(sorted_clstr) as infile:
        for line in infile.readlines():
            line = line.strip()
            if line:
                if line.startswith('>'):
                    cid = int(line.split()[1]) + start_clst
                    # clst_id[cid] = {}
                else:
                    alst = line.split()
                    pid = re.sub('^>','',alst[2])
                    pid = re.sub('...$','',pid)
                    if alst[3] == "*":
                        # v = 2.
                        clst_rep[cid] = pid
                    # else:
                        # v = float(re.sub('%','',alst[-1]))/100
                    # clst_id[cid][pid] = v
                    pro_cid[pid] = cid
                    
    # return pro_cid,clst_id,clst_rep
    return pro_cid, clst_rep

In [4]:
## process protein clusters
pro_cid_90,clst_rep_90 = get_cluster_id('Analysis/cdhit/pro90.sorted.clstr')
pro_cid_80,clst_rep_80 = get_cluster_id('Analysis/cdhit/pro80.sorted.clstr') 

s_pro_cid_90,s_clst_rep_90 = get_cluster_id('Analysis/cdhit/pro90.short.sorted.clstr', start_clst=max(clst_rep_90.keys()))
s_pro_cid_80,s_clst_rep_80 = get_cluster_id('Analysis/cdhit/pro80.short.sorted.clstr', start_clst=max(clst_rep_80.keys()))                        

pro_cid_90 = pro_cid_90 | s_pro_cid_90
clst_rep_90 = clst_rep_90 | s_clst_rep_90
pro_cid_80 = pro_cid_80 | s_pro_cid_80
clst_rep_80 = clst_rep_80 | s_clst_rep_80

data['Pro_clst'] = [ pro_cid_90[x] if x in pro_cid_90.keys() else -1 for x in data.ID.tolist() ]
data['Pro_clst_rep'] = [ clst_rep_90[x] if x in clst_rep_90.keys() else 'NoData' for x in data.Pro_clst.tolist() ]
data['Pro_clst80'] = [ pro_cid_80[x] if x in pro_cid_80.keys() else -1 for x in data['Pro_clst_rep'].tolist() ]
data['Pro_clst80_rep'] = [ clst_rep_80[x] if x in clst_rep_80.keys() else 'NoData' for x in data['Pro_clst80'].tolist() ]

data.sort_values(by=['AMP_clst','Pro_clst80','Pro_clst']).to_csv('rmdup.animal.clst90_80.AMPclst95.eggnog2.txt',sep='\t',index=False)

In [69]:
## process amp clusters
amp_cid_95,aclst_rep_95 = get_cluster_id('Analysis/amp_cdhit/amp95.sorted.clstr')
s_amp_cid_95,s_aclst_rep_95 = get_cluster_id('Analysis/amp_cdhit/amp95.short.sorted.clstr', start_clst=max(aclst_rep_95.keys()))

amp_cid_95 = amp_cid_95 | s_amp_cid_95

data['AMP_clst'] = [ amp_cid_95[x] if x in amp_cid_95.keys() else -1 for x in data.AMPID.tolist() ]

#### Step 4, Get eggnog-mapper annotation

output = 'all_amp_protein.animal.clst90_80.AMP_clst95.eggnog.txt'

In [70]:
def rm_X_pro(df):
    df['X_flag'] = [ 1 if not 'X' in x else 0 for x in df['Sequence'].tolist() ]
    df = df[df.X_flag==1]
    del df['X_flag']
    return df

print('Before removing X sequence:')
print(data.shape)

data = rm_X_pro(data)
print('After removing X sequence:')
print(data.shape)

Before removing X sequence:
(5764503, 12)
After removing X sequence:
(5762475, 12)


In [71]:
egg = pd.read_csv('eggnogMapper_fg/eggnog_mapper.txt',sep='\t')
egg = egg.drop_duplicates()

In [ ]:
data2egg = pd.merge(data, egg, left_on='Pro_clst_rep', right_on='query', how='left')
data2egg['seed_ortholog'].fillna('NoData',inplace=True)
data2egg.fillna('-',inplace=True)
del data2egg['query']

data2egg.to_csv('all_amp_protein.animal.clst90_80.AMPclst95.eggnog.txt',sep='\t',index=False)

In [48]:
data2egg_rmdup = data2egg.drop_duplicates(subset=['Source','Sequence','AMP','Position'])
print('After removing duplicates:')
print(data2egg_rmdup.shape)
data2egg_rmdup.to_csv('rmdup.animal.clst90_80.AMPclst95.eggnog.txt',sep='\t',index=False)

After removing duplicates:
(1522945, 32)


#### Step 5, Statistics on AMP- & Pro- clusters

| Output | Description |
| - | - |
| 'rmdup.animal.clst90_80.AMPclst95.eggnog.filtered.txt' | All filtered AMPs & Proteins |
| 'rmdup.animal.clst90_80.AMPclst95.eggnog.filtered.amp_repr_stat.txt' | Statistics on AMP-clusters |
| 'rmdup.animal.clst90_80.AMPclst95.eggnog.filtered.repr_stat.txt' | Staticstics on Protein-clusters |

In [49]:
pro_path = 'Clustering80_rmdup/Protein'
amp_path = 'Clustering80_rmdup/AMPfragment'

In [50]:
def calc_source_count(df):
    _dict = df['Source'].value_counts().to_dict()
    total = sum(_dict.values())
    ## AMP counts (after duplicates-removed)
    orals = ['MGnify_human_oral','BGI_human_oral']
    animals = ['MGnify_cow_rumen', 'MGnify_pig_gut', 'MGnify_zibrafish_fecal','MGnify_fish_gut']
    human_gut_total = 0
    modern_human_gut = 0
    ancient_human_gut = 0
    oral_counts = 0
    animal_counts = 0
    
    for g in ['data_CGMR','MGnify_human_gut']:
        if g in _dict.keys():
            human_gut_total += _dict[g]
            modern_human_gut += _dict[g]
    
    for g in ['data_Hadza','ancient_human_gut']:
        if g in _dict.keys():
            human_gut_total += _dict[g]
            ancient_human_gut += _dict[g]
    
    for o in orals:
        for o in _dict.keys():
            oral_counts += _dict[o]
    
    for a in animals:
        for a in _dict.keys():
            animal_counts += _dict[a]
    
    return total, human_gut_total, modern_human_gut, ancient_human_gut, oral_counts, animal_counts

def output_clusters(clstr_id, df, pro_path, amp_path):
    ids = df['ID'].tolist()
    seqs = df['Sequence'].tolist()
    amps = df['AMP'].tolist()

    with open(os.path.join(pro_path, clstr_id+'.faa'), 'w') as pout:
        with open(os.path.join(amp_path, clstr_id+'.faa'), 'w') as aout:
            for i in range(len(ids)):
                pout.write('>%s\n%s\n' % (ids[i],seqs[i]))
                aout.write('>%s\n%s\n' % (ids[i],amps[i]))